In [1]:
# <-- Import libraries, custom functions, and load configuration & datasets <-- #

import yaml
import datetime as dt
import re
import pandas as pd
import scipy.stats as st
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import statsmodels.api as sm
import seaborn as sns

from statsmodels.multivariate.manova import MANOVA
from scipy import stats
from scipy.stats import pearsonr, boxcox, chi2_contingency, shapiro, probplot
from scipy.stats.contingency import association

sns.set(style="whitegrid")

# <-- Imports custom preprocessing functions from 'functions.py' <-- #

# from functions import ()

# <-- Loads YAML configuration to dynamically reference CSV output files. <-- #

config = None  # <-- Initialize config
try:
    with open("../config.yaml", "r") as file:
        config = yaml.safe_load(file)
except:
    print("Yaml configuration file not found!")

df_demo_test = pd.read_csv(config['output_data']['file3'])          # demo_test clients
df_demo_control = pd.read_csv(config['output_data']['file4'])       # demo_control clients
df_web_data = pd.read_csv(config['output_data']['file5'])           # web_data clean
df_web_data_control = pd.read_csv(config['output_data']['file6'])   # web_data demo_control clients (for Sanity checks)
df_web_data_test = pd.read_csv(config['output_data']['file7'])      # web_data demo_test clients (for Sanity checks)


In [2]:
df_demo_control.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23526 entries, 0 to 23525
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   client_id          23526 non-null  int64  
 1   clnt_tenure_yr     23526 non-null  float64
 2   clnt_tenure_mnth   23526 non-null  float64
 3   clnt_age           23526 non-null  float64
 4   num_accts          23526 non-null  float64
 5   bal                23526 non-null  float64
 6   calls_6_mnth       23526 non-null  float64
 7   logons_6_mnth      23526 non-null  float64
 8   Variation          23526 non-null  object 
 9   clnt_age_quantile  23526 non-null  object 
 10  tenure_quantile    23526 non-null  object 
dtypes: float64(7), int64(1), object(3)
memory usage: 2.0+ MB


In [3]:
df_demo_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26961 entries, 0 to 26960
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   client_id          26961 non-null  int64  
 1   clnt_tenure_yr     26961 non-null  float64
 2   clnt_tenure_mnth   26961 non-null  float64
 3   clnt_age           26961 non-null  float64
 4   num_accts          26961 non-null  float64
 5   bal                26961 non-null  float64
 6   calls_6_mnth       26961 non-null  float64
 7   logons_6_mnth      26961 non-null  float64
 8   Variation          26961 non-null  object 
 9   clnt_age_quantile  26961 non-null  object 
 10  tenure_quantile    26961 non-null  object 
dtypes: float64(7), int64(1), object(3)
memory usage: 2.3+ MB


In [4]:
# ========= Config =========
STEP_ORDER = ['start','step_1','step_2','step_3','confirm']
STEP_MAP   = {s:i for i,s in enumerate(STEP_ORDER)}
KEY = ['client_id','visitor_id','visit_id']   # key to the process

def _to_utc(series: pd.Series) -> pd.Series:
    s = pd.to_datetime(series, errors='coerce')
    if getattr(s.dt, "tz", None) is None:
        return s.dt.tz_localize("UTC")
    return s.dt.tz_convert("UTC")

def _standardize_steps(df):
    w = df.copy()
    w['process_step'] = (w['process_step'].str.lower().str.strip()
                         .str.replace(' ', '_')
                         .str.replace('step1','step_1')
                         .str.replace('step2','step_2')
                         .str.replace('step3','step_3'))
    return w[w['process_step'].isin(STEP_ORDER)]


In [5]:
# ========= 1) Filtered to the group and normalization =========
def prepare_web_for_group(df_web_data, df_demo_group):
    ids = set(df_demo_group['client_id'].unique())
    w = df_web_data[df_web_data['client_id'].isin(ids)].copy()
    w = _standardize_steps(w)
    w['date_time'] = _to_utc(w['date_time'])
    # Order **by process**
    w = w.sort_values(KEY + ['date_time']).reset_index(drop=True)
    return w

In [6]:
# ========= 2) Complete sequence → step-backs =========
def compute_back_jumps(w_full: pd.DataFrame) -> pd.DataFrame:
    wf = w_full.copy()
    wf['step_idx'] = wf['process_step'].map(STEP_MAP).astype('Int64')
    wf['prev_step_idx'] = wf.groupby(KEY)['step_idx'].shift(1)
    wf['delta'] = wf['step_idx'] - wf['prev_step_idx']
    wf['is_back_jump'] = wf['delta'].lt(0)
    back = (wf.groupby(KEY)
              .agg(n_back_jumps=('is_back_jump','sum'))
              .reset_index())
    return wf, back

In [7]:
# ========= 3) Last occurrences per step and last confirm =========
def collapse_last_per_step_and_last_confirm(wf: pd.DataFrame) -> pd.DataFrame:
    '''
    For each process (KEY):
      - Keep only the *last* confirm (drop intermediate confirms).
      - Keep the *first* start.
      - Keep the *last* for step_1, step_2, step_3.
    Returns a compact event table with at most one row per (process, step).
    '''
    # 1) Keep only the last confirm per process
    is_last_confirm = (wf['process_step'].eq('confirm') & 
                       ~wf.duplicated(subset=KEY + ['process_step'], keep='last'))
    drop_mask = wf['process_step'].eq('confirm') & (~is_last_confirm)
    wf_no_mid_conf = wf.loc[~drop_mask].copy()
    
    # 2) Split into start vs others
    start_df  = wf_no_mid_conf[wf_no_mid_conf['process_step'].eq('start')].copy()
    other_df  = wf_no_mid_conf[~wf_no_mid_conf['process_step'].eq('start')].copy()

    # 3) For start: keep the *first* occurrence per process
    #    (sorted by time so "first" truly means earliest)
    start_first = (
        start_df.sort_values(KEY + ['date_time'])
                .drop_duplicates(subset=KEY + ['process_step'], keep='first')
    )

    # 4) For others (step_1/step_2/step_3/confirm): keep the *last* occurrence per step
    other_last = (
        other_df.sort_values(KEY + ['process_step', 'date_time'])
                .drop_duplicates(subset=KEY + ['process_step'], keep='last')
    )

    # 5) Combine back and keep a tidy ordering
    wf2 = (
        pd.concat([start_first, other_last], ignore_index=True)
          .sort_values(KEY + ['process_step', 'date_time'])
          .reset_index(drop=True)
    )

    return wf2
    
    # # We remove intermediate confirms per process (leaving only the last one)
    # is_last_confirm = (wf['process_step'].eq('confirm') &
    #                    ~wf.duplicated(subset=KEY+['process_step'], keep='last'))
    # drop_mask = wf['process_step'].eq('confirm') & (~is_last_confirm)
    # wf2 = wf.loc[~drop_mask].copy()

    # # For completion times we take the **last** occurrence of each step per process
    # wf2 = (wf2.sort_values(KEY + ['process_step','date_time'])
    #           .drop_duplicates(subset=KEY+['process_step'], keep='last'))
    # return wf2

In [8]:
# ========= 4) Table by process (1 row = 1 process) =========
def summarize_processes(wf2: pd.DataFrame, back: pd.DataFrame) -> pd.DataFrame:
    # Pivot de timestamps
    pv = wf2.pivot_table(index=KEY, columns='process_step', values='date_time', aggfunc='last') \
            .reindex(columns=STEP_ORDER)
    for c in STEP_ORDER:
        if c not in pv.columns:
            pv[c] = pd.NaT
    # all as naive UTC for subtractions
    for c in pv.columns:
        pv[c] = _to_utc(pv[c]).dt.tz_convert('UTC').dt.tz_localize(None)

    # flags reached and completed
    flags = pv.notna().astype(int).add_prefix('reached_')
    flags['completed'] = flags['reached_confirm']

    # times (min)
    def mins(b,a): return (pv[b] - pv[a]) / pd.Timedelta(minutes=1)
    times = pd.DataFrame(index=pv.index)
    times['t_start_step1'] = mins('step_1','start')
    times['t_step1_step2'] = mins('step_2','step_1')
    times['t_step2_step3'] = mins('step_3','step_2')
    times['t_step3_conf']  = mins('confirm','step_3')
    times['t_total']       = mins('confirm','start')

    proc = (flags.join(times)).reset_index()
    proc = proc.merge(back, on=KEY, how='left').fillna({'n_back_jumps':0})

    # Outcome:
    # - successful: completed and no step-backs
    # - completed_with_errors: completed with step-backs
    # - fail: no completed
    proc['outcome'] = np.where(
        (proc['completed']==1) & (proc['n_back_jumps']==0), 'successful',
        np.where((proc['completed']==1) & (proc['n_back_jumps']>0), 'completed_with_errors', 'fail')
    )
    return proc

In [9]:
# ========= 5) KPIs & Drop-off =========
def kpis_from_processes(proc: pd.DataFrame) -> pd.DataFrame:
    d = proc.copy()
    denom = max(1, int(d['reached_start'].sum()))   # who started

    kpis = {
        'n_processes'          : int(len(d)),
        'started'              : int(d['reached_start'].sum()),
        'step1_rate_%'         : 100*d['reached_step_1'].sum()/denom,
        'step2_rate_%'         : 100*d['reached_step_2'].sum()/denom,
        'step3_rate_%'         : 100*d['reached_step_3'].sum()/denom,
        'completion_rate_%'    : 100*d['completed'].sum()/denom,
        'successful_%'              : 100*(d['outcome'].eq('successful').sum())/denom,
        'completed_with_errors_%'   : 100*(d['outcome'].eq('completed_with_errors').sum())/denom,
        'fail%'                     : 100*(d['outcome'].eq('fail').sum())/denom,
        't_total_avg_min'   : float(np.nanmean(d['t_total'])),
        # 't_total_median_min'   : float(np.nanmedian(d['t_total'])),
        't_step1_avg_min'   : float(np.nanmean(d['t_start_step1'])),
        # 't_step1_median_min'   : float(np.nanmedian(d['t_start_step1'])),
        't_step2_avg_min'   : float(np.nanmean(d['t_step1_step2'])),
        # 't_step2_median_min'   : float(np.nanmedian(d['t_step1_step2'])),
        't_step3_avg_min'   : float(np.nanmean(d['t_step2_step3'])),
        # 't_step3_median_min'   : float(np.nanmedian(d['t_step2_step3'])),
        't_conf_avg_min'    : float(np.nanmean(d['t_step3_conf'])),
        # 't_conf_median_min'    : float(np.nanmedian(d['t_step3_conf'])),
        'n_back_jumps'       : int(d['n_back_jumps'].sum()),
        'avg_back_jumps'       : float(d['n_back_jumps'].mean())
    }
    return pd.DataFrame([kpis])

In [10]:
def step_dropoff_table(proc: pd.DataFrame) -> pd.DataFrame:
    d = proc.copy()
    rows = [
        ("start→step_1", int(d['reached_start'].sum()),   int(d['reached_step_1'].sum())),
        ("step_1→step_2", int(d['reached_step_1'].sum()), int(d['reached_step_2'].sum())),
        ("step_2→step_3", int(d['reached_step_2'].sum()), int(d['reached_step_3'].sum())),
        ("step_3→confirm", int(d['reached_step_3'].sum()), int(d['completed'].sum())),
    ]
    out = pd.DataFrame(rows, columns=['from_to','n_from','n_to'])
    out['conv_rate_%'] = 100 * out['n_to'] / out['n_from'].replace({0:np.nan})
    out['conv_rate_%'] = out['conv_rate_%'].fillna(0.0)
    out['dropoff_%'] = 100 - out['conv_rate_%']
    return out

In [11]:
def stranger_things(proc_df: pd.DataFrame, key_cols=(KEY)):
    '''
    Cleans up funnel inconsistencies in an already summarized process DataFrame (e.g., proc_control).
    Rules:
    - completed==1 but reached_start==0
    - Step reached with previous steps not reached.
    Returns:
    proc_clean, proc_anomalies
    '''
    df = proc_df.copy()

    # Ensures required columns (if missing, creates them with 0)
    needed = ['reached_start','reached_step_1','reached_step_2','reached_step_3','reached_confirm','completed','t_total']
    for c in needed:
        if c not in df.columns:
            df[c] = 0

    # Masks of inconsistencies
    m_step1_prev = (df['reached_step_1'].eq(1) & df['reached_start'].eq(0))
    m_step2_prev = (df['reached_step_2'].eq(1) &
                    ((df['reached_start'].eq(0)) | (df['reached_step_1'].eq(0))))

    m_step3_prev = (df['reached_step_3'].eq(1) &
                    ((df['reached_start'].eq(0)) | (df['reached_step_1'].eq(0)) | (df['reached_step_2'].eq(0))))

    m_conf_prev  = (df['reached_confirm'].eq(1) &
                    ((df['reached_start'].eq(0)) | (df['reached_step_1'].eq(0)) | (df['reached_step_2'].eq(0)) | (df['reached_step_3'].eq(0))))
    
    m_not_start  = (df['t_total'] < 0)
    
    # Full mask
    m_bad = m_step1_prev | m_step2_prev | m_step3_prev | m_conf_prev | m_not_start

    # Main reason (priority by order)
    reason = np.select(
        [
            m_conf_prev,
            m_step3_prev,
            m_step2_prev,
            m_step1_prev,
            m_not_start
        ],
        [
            "reached_confirm with missing previous step(s)",
            "reached_step_3 with missing previous step(s)",
            "reached_step_2 with missing previous step(s)",
            "reached_step_1 with reached_start==0",
            "first chronological event != start"
        ],
        default=""
    )

    df['reason'] = reason

    # Separates anomalies and cleans
    proc_anomalies = df.loc[m_bad].copy()
    proc_clean = df.loc[~m_bad].drop(columns=['reason'])

    # # Ordena columnas (optional): keys first
    # if all(k in proc_clean.columns for k in key_cols):
    #     front = list(key_cols)
    #     proc_clean = proc_clean[front + [c for c in proc_clean.columns if c not in front]]
    #     if not proc_anomalies.empty:
    #         proc_anomalies = proc_anomalies[front + [c for c in proc_anomalies.columns if c not in front]]

    return proc_clean.reset_index(drop=True), proc_anomalies.reset_index(drop=True)


In [12]:
def set_negative_to_nan(df: pd.DataFrame, cols=['t_start_step1', 't_step1_step2', 't_step2_step3'], inplace: bool = False) -> pd.DataFrame:
    """
    For each column in `cols`, if value < 0 -> NaN; else keep value.
    Ensures the column is numeric first.
    """
    d = df if inplace else df.copy()
    for c in ['t_start_step1','t_step1_step2','t_step2_step3','t_step3_conf']:
        d[c] = pd.to_numeric(d[c], errors='coerce').apply(lambda x: np.nan if pd.notna(x) and x < 0 else x)
    return d

In [13]:
# ========= 6) RUN (CONTROL) =========
w_control = prepare_web_for_group(df_web_data, df_demo_control)               # filters and normalizes
wf, back = compute_back_jumps(w_control)                                      # complete sequence → step-backs
wf2 = collapse_last_per_step_and_last_confirm(wf)                             # collapses to last occurrence per step
proc_control = summarize_processes(wf2, back)                                 # 11 row per process + outcomee
proc_control_clean, proc_control_anomalies = stranger_things(proc_control)
proc_control_clean = set_negative_to_nan(proc_control_clean)
kpis_control = kpis_from_processes(proc_control_clean)                        # KPIs
dropoff_control = step_dropoff_table(proc_control_clean)

display(proc_control_clean[['client_id', 'visitor_id', 'visit_id','reached_start','reached_step_1', 'reached_step_2', 'reached_step_3', 'reached_confirm','completed']].head(5))
display(proc_control_clean[['client_id', 'visitor_id', 'visit_id','t_start_step1', 't_step1_step2', 't_step2_step3','t_step3_conf', 't_total', 'n_back_jumps', 'outcome']].head(5))
# display(proc_control_clean.head(15))
display(kpis_control[['n_processes', 'started', 'step1_rate_%', 'step2_rate_%','step3_rate_%', 'completion_rate_%', 'successful_%', 'completed_with_errors_%', 'fail%']])
display(kpis_control[['t_total_avg_min', 't_step1_avg_min', 't_step2_avg_min', 't_step3_avg_min','t_conf_avg_min', 'n_back_jumps', 'avg_back_jumps']])
# display(kpis_control)
display(dropoff_control)

print("Valid processes:", len(proc_control_clean))
print("Processes driven by inconsistencies:", len(proc_control_anomalies))
# display(proc_control_anomalies.head(5))

# Count distinct control clients represented in web data
print("Distinct control clients with activity:", proc_control_clean['client_id'].nunique())

# Save files
proc_control_clean.to_csv(config['output_data']['file8'], index=False, sep=",", encoding="utf-8")
proc_control_anomalies.to_csv(config['output_data']['file10'], index=False, sep=",", encoding="utf-8")

print(f'\n======================================================================')
print(f' ***   Sanity Checks:')
print(f'======================================================================')

client_id=[81976]
# client_id=[4079649, 4033851, 5305116, 1028, 69255, 613664, 21970, 81976]

for id in (client_id):
      display(f"Client {id} summarize_processes:", 
            proc_control_clean[proc_control_clean['client_id'] == id])
      
      display(f"Client {id} processes before collapse_last_per_step_and_last_confirm:", 
            wf[wf['client_id'] == id])
      
      display(f"Client {id} processes after collapse_last_per_step_and_last_confirm:", 
            wf2[wf2['client_id'] == id].sort_values(by=['date_time'],ascending=True))
      
      # display(f"Client {id} processes:", 
      #       proc_control_clean[proc_control_clean['client_id']==id][KEY+['completed','n_back_jumps','outcome','t_total']])
      
      display(f"Client {id} processes in raw we_data:", 
            df_web_data_control[df_web_data_control['client_id'] == id].sort_values(by='date_time'))
            

,client_id,visitor_id,visit_id,reached_start,reached_step_1,reached_step_2,reached_step_3,reached_confirm,completed
0,1028,42237450_62128060588,557292053_87239438319_391157,1,1,1,1,0,0
1,1186,446844663_31615102958,507052512_11309370126_442139,1,0,0,0,0,0
2,1186,446844663_31615102958,795373564_99931517312_810896,1,1,1,0,0,0
3,1195,766842522_69992551638,393817425_39015278493_996341,1,1,1,1,1,1
4,1197,753759429_54481946928,71862471_21202285428_848395,1,1,1,1,1,1


,client_id,visitor_id,visit_id,t_start_step1,t_step1_step2,t_step2_step3,t_step3_conf,t_total,n_back_jumps,outcome
0,1028,42237450_62128060588,557292053_87239438319_391157,8.966667,NaN,NaN,NaN,NaN,2,fail
1,1186,446844663_31615102958,507052512_11309370126_442139,NaN,NaN,NaN,NaN,NaN,0,fail
2,1186,446844663_31615102958,795373564_99931517312_810896,0.183333,0.183333,NaN,NaN,NaN,0,fail
3,1195,766842522_69992551638,393817425_39015278493_996341,0.550000,1.633333,0.516667,1.383333,4.083333,0,successful
4,1197,753759429_54481946928,71862471_21202285428_848395,0.066667,1.133333,0.100000,0.283333,1.583333,1,completed_with_errors


,n_processes,started,step1_rate_%,step2_rate_%,step3_rate_%,completion_rate_%,successful_%,completed_with_errors_%,fail%
0,22819,22819,74.51685,63.324423,57.254919,48.012621,36.776371,11.23625,51.987379


,t_total_avg_min,t_step1_avg_min,t_step2_avg_min,t_step3_avg_min,t_conf_avg_min,n_back_jumps,avg_back_jumps
0,6.785173,2.222788,0.925299,1.607815,2.339446,5948,0.26066


,from_to,n_from,n_to,conv_rate_%,dropoff_%
0,start→step_1,22819,17004,74.516850,25.483150
1,step_1→step_2,17004,14450,84.980005,15.019995
2,step_2→step_3,14450,13065,90.415225,9.584775
3,step_3→confirm,13065,10956,83.857635,16.142365


...

,client_id,visitor_id,visit_id,process_step,date_time,step_idx,prev_step_idx,delta,is_back_jump
860,81976,651554010_72966877074,352605520_96472725570_616578,step_2,2017-04-14 12:57:26+00:00,2,<NA>,<NA>,<NA>
861,81976,651554010_72966877074,352605520_96472725570_616578,step_3,2017-04-14 12:59:39+00:00,3,2,1,False
862,81976,651554010_72966877074,352605520_96472725570_616578,confirm,2017-04-14 13:02:30+00:00,4,3,1,False
863,81976,651554010_72966877074,388361902_83848416362_286403,start,2017-04-14 12:55:38+00:00,0,<NA>,<NA>,<NA>
864,81976,651554010_72966877074,388361902_83848416362_286403,step_1,2017-04-14 12:56:27+00:00,1,0,1,False


'Client 81976 processes after collapse_last_per_step_and_last_confirm:'

,client_id,visitor_id,visit_id,process_step,date_time,step_idx,prev_step_idx,delta,is_back_jump
692,81976,651554010_72966877074,388361902_83848416362_286403,start,2017-04-14 12:55:38+00:00,0,<NA>,<NA>,<NA>
693,81976,651554010_72966877074,388361902_83848416362_286403,step_1,2017-04-14 12:56:27+00:00,1,0,1,False
690,81976,651554010_72966877074,352605520_96472725570_616578,step_2,2017-04-14 12:57:26+00:00,2,<NA>,<NA>,<NA>
691,81976,651554010_72966877074,352605520_96472725570_616578,step_3,2017-04-14 12:59:39+00:00,3,2,1,False
689,81976,651554010_72966877074,352605520_96472725570_616578,confirm,2017-04-14 13:02:30+00:00,4,3,1,False


'Client 81976 processes in raw we_data:'

,client_id,visitor_id,visit_id,process_step,date_time
76189,81976,651554010_72966877074,388361902_83848416362_286403,start,2017-04-14 12:55:38
76193,81976,651554010_72966877074,388361902_83848416362_286403,step_1,2017-04-14 12:56:27
76198,81976,651554010_72966877074,352605520_96472725570_616578,step_2,2017-04-14 12:57:26
76205,81976,651554010_72966877074,352605520_96472725570_616578,step_3,2017-04-14 12:59:39
76211,81976,651554010_72966877074,352605520_96472725570_616578,confirm,2017-04-14 13:02:30


In [14]:
# proc_control_clean[proc_control_clean['t_step3_conf']<0]
mask = (wf['step_idx'].ne(0)) & (wf['prev_step_idx'].isna())
rows = wf.loc[mask]
rows

,client_id,visitor_id,visit_id,process_step,date_time,step_idx,prev_step_idx,delta,is_back_jump
225,21970,884689567_69949404962,579369959_76319820707_173529,step_3,2017-04-09 19:34:40+00:00,3,<NA>,<NA>,<NA>
705,69255,310869805_9693330193,749978437_90205781818_231452,step_3,2017-04-05 14:44:43+00:00,3,<NA>,<NA>,<NA>
860,81976,651554010_72966877074,352605520_96472725570_616578,step_2,2017-04-14 12:57:26+00:00,2,<NA>,<NA>,<NA>
1024,89108,365572967_15193040764,387704599_68740390385_217814,step_2,2017-04-14 15:24:23+00:00,2,<NA>,<NA>,<NA>
1228,117979,374753105_98088955424,424224737_67847811518_581313,step_2,2017-04-05 12:05:05+00:00,2,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...
99385,9868880,445608976_36629464124,765368868_94020730286_151207,step_3,2017-04-05 13:30:19+00:00,3,<NA>,<NA>,<NA>
99405,9871504,533467360_75772722838,500445787_67477918703_690686,step_1,2017-04-09 14:47:40+00:00,1,<NA>,<NA>,<NA>
99450,9876208,882766623_91160541768,992225799_22633045400_643858,step_1,2017-04-09 16:40:25+00:00,1,<NA>,<NA>,<NA>
99704,9900228,853691791_94850916729,362053301_52900309373_354088,step_1,2017-04-09 17:11:45+00:00,1,<NA>,<NA>,<NA>


In [17]:
# ========= 6) RUN (TEST) =========
w_test = prepare_web_for_group(df_web_data, df_demo_test)               # filters and normalizes
wf_test, back = compute_back_jumps(w_test)                              # complete sequence → step-backs
wf2_test = collapse_last_per_step_and_last_confirm(wf_test)             # collapses to last occurrence per step
proc_test = summarize_processes(wf2_test, back)                         # 1 row per process + outcomee
proc_test_clean, proc_test_anomalies = stranger_things(proc_test)
proc_test_clean = set_negative_to_nan(proc_test_clean)
kpis_test = kpis_from_processes(proc_test_clean)                        # KPIs
dropoff_test = step_dropoff_table(proc_test_clean)

display(proc_test_clean[['client_id', 'visitor_id', 'visit_id','reached_start','reached_step_1', 'reached_step_2', 'reached_step_3', 'reached_confirm','completed']].head(5))
display(proc_test_clean[['client_id', 'visitor_id', 'visit_id','t_start_step1', 't_step1_step2', 't_step2_step3','t_step3_conf', 't_total', 'n_back_jumps', 'outcome']].head(5))
# display(proc_test_clean.head(15))
display(kpis_test[['n_processes', 'started', 'step1_rate_%', 'step2_rate_%','step3_rate_%', 'completion_rate_%', 'successful_%', 'completed_with_errors_%', 'fail%']])
display(kpis_test[['t_total_avg_min', 't_step1_avg_min', 't_step2_avg_min', 't_step3_avg_min','t_conf_avg_min', 'n_back_jumps', 'avg_back_jumps']])
# display(kpis_test)
display(dropoff_test)

print("Valid processes:", len(proc_test_clean))
print("Processes driven by inconsistencies:", len(proc_test_anomalies))
# display(proc_test_anomalies.head(5))

# Count distinct test clients represented in web data
print("Distinct test clients with activity:", proc_test_clean['client_id'].nunique())

# Save files
proc_test_clean.to_csv(config['output_data']['file9'], index=False, sep=",", encoding="utf-8")
proc_test_anomalies.to_csv(config['output_data']['file11'], index=False, sep=",", encoding="utf-8")

print(f'\n======================================================================')
print(f' ***   Sanity Checks:')
print(f'======================================================================')

client_id=[4600680]
# client_id=[7230446, 555, 984576]

for id in (client_id):
      display(f"Client {id} summarize_processes:", 
            proc_test_clean[proc_test_clean['client_id'] == id])
      
      display(f"Client {id} processes before collapse_last_per_step_and_last_confirm:", 
            wf_test[wf_test['client_id'] == id].sort_values(by=['date_time'],ascending=True))
      
      display(f"Client {id} processes after collapse_last_per_step_and_last_confirm:", 
            wf2_test[wf2_test['client_id'] == id].sort_values(by=['date_time'],ascending=True))
      
      # display(f"Client {id} processes:", 
      #       proc_test_clean[proc_test_clean['client_id']==id][KEY+['completed','n_back_jumps','outcome','t_total']])
      
      display(f"Client {id} processes in raw we_data:", 
            df_web_data_test[df_web_data_test['client_id'] == id].sort_values(by='date_time'))

,client_id,visitor_id,visit_id,reached_start,reached_step_1,reached_step_2,reached_step_3,reached_confirm,completed
0,555,402506806_56087378777,637149525_38041617439_716659,1,1,1,1,1,1
1,647,66758770_53988066587,40369564_40101682850_311847,1,1,1,1,1,1
2,934,810392784_45004760546,7076463_57954418406_971348,1,0,0,0,0,0
3,1516,182314299_63168583136,255400977_38039535960_779641,1,1,1,1,1,1
4,1516,182314299_63168583136,481123290_95510977345_707323,1,1,1,0,0,0


,client_id,visitor_id,visit_id,t_start_step1,t_step1_step2,t_step2_step3,t_step3_conf,t_total,n_back_jumps,outcome
0,555,402506806_56087378777,637149525_38041617439_716659,0.116667,0.533333,1.650000,0.333333,2.633333,0,successful
1,647,66758770_53988066587,40369564_40101682850_311847,0.116667,0.300000,3.150000,2.716667,6.283333,0,successful
2,934,810392784_45004760546,7076463_57954418406_971348,NaN,NaN,NaN,NaN,NaN,0,fail
3,1516,182314299_63168583136,255400977_38039535960_779641,0.933333,0.583333,10.116667,4.266667,15.900000,0,successful
4,1516,182314299_63168583136,481123290_95510977345_707323,4.083333,NaN,NaN,NaN,NaN,3,fail


,n_processes,started,step1_rate_%,step2_rate_%,step3_rate_%,completion_rate_%,successful_%,completed_with_errors_%,fail%
0,25932,25932,85.947864,74.332871,66.894185,55.360173,42.333796,13.026377,44.639827


,t_total_avg_min,t_step1_avg_min,t_step2_avg_min,t_step3_avg_min,t_conf_avg_min,n_back_jumps,avg_back_jumps
0,6.984293,2.573227,0.873905,1.664498,2.463916,12422,0.479022


,from_to,n_from,n_to,conv_rate_%,dropoff_%
0,start→step_1,25932,22288,85.947864,14.052136
1,step_1→step_2,22288,19276,86.486001,13.513999
2,step_2→step_3,19276,17347,89.992737,10.007263
3,step_3→confirm,17347,14356,82.757826,17.242174


...

,client_id,visitor_id,visit_id,process_step,date_time,step_idx,prev_step_idx,delta,is_back_jump
63809,4600680,208827041_70541462157,935958091_89261622321_315288,start,2017-04-05 21:31:00+00:00,0,<NA>,<NA>,<NA>
63808,4600680,208827041_70541462157,672737966_14726163487_971608,start,2017-04-08 08:40:30+00:00,0,<NA>,<NA>,<NA>
63800,4600680,208827041_70541462157,571402946_85481266703_66292,step_1,2017-04-08 09:12:37+00:00,1,<NA>,<NA>,<NA>
63801,4600680,208827041_70541462157,571402946_85481266703_66292,step_2,2017-04-08 09:12:49+00:00,2,1,1,False
63802,4600680,208827041_70541462157,571402946_85481266703_66292,step_3,2017-04-08 09:16:37+00:00,3,2,1,False
63803,4600680,208827041_70541462157,571402946_85481266703_66292,step_2,2017-04-08 09:20:56+00:00,2,3,-1,True
63804,4600680,208827041_70541462157,571402946_85481266703_66292,step_3,2017-04-08 09:21:04+00:00,3,2,1,False
63805,4600680,208827041_70541462157,571402946_85481266703_66292,confirm,2017-04-08 09:23:40+00:00,4,3,1,False
63806,4600680,208827041_70541462157,571402946_85481266703_66292,start,2017-04-08 09:28:21+00:00,0,4,-4,True
63807,4600680,208827041_70541462157,571402946_85481266703_66292,step_1,2017-04-08 09:28:23+00:00,1,0,1,False


'Client 4600680 processes after collapse_last_per_step_and_last_confirm:'

,client_id,visitor_id,visit_id,process_step,date_time,step_idx,prev_step_idx,delta,is_back_jump
47361,4600680,208827041_70541462157,935958091_89261622321_315288,start,2017-04-05 21:31:00+00:00,0,<NA>,<NA>,<NA>
47360,4600680,208827041_70541462157,672737966_14726163487_971608,start,2017-04-08 08:40:30+00:00,0,<NA>,<NA>,<NA>
47358,4600680,208827041_70541462157,571402946_85481266703_66292,step_2,2017-04-08 09:20:56+00:00,2,3,-1,True
47359,4600680,208827041_70541462157,571402946_85481266703_66292,step_3,2017-04-08 09:21:04+00:00,3,2,1,False
47355,4600680,208827041_70541462157,571402946_85481266703_66292,confirm,2017-04-08 09:23:40+00:00,4,3,1,False
47356,4600680,208827041_70541462157,571402946_85481266703_66292,start,2017-04-08 09:28:21+00:00,0,4,-4,True
47357,4600680,208827041_70541462157,571402946_85481266703_66292,step_1,2017-04-08 09:28:23+00:00,1,0,1,False


'Client 4600680 processes in raw we_data:'

,client_id,visitor_id,visit_id,process_step,date_time
63279,4600680,208827041_70541462157,935958091_89261622321_315288,start,2017-04-05 21:31:00
73395,4600680,208827041_70541462157,672737966_14726163487_971608,start,2017-04-08 08:40:30
73474,4600680,208827041_70541462157,571402946_85481266703_66292,step_1,2017-04-08 09:12:37
73475,4600680,208827041_70541462157,571402946_85481266703_66292,step_2,2017-04-08 09:12:49
73482,4600680,208827041_70541462157,571402946_85481266703_66292,step_3,2017-04-08 09:16:37
73491,4600680,208827041_70541462157,571402946_85481266703_66292,step_2,2017-04-08 09:20:56
73492,4600680,208827041_70541462157,571402946_85481266703_66292,step_3,2017-04-08 09:21:04
73504,4600680,208827041_70541462157,571402946_85481266703_66292,confirm,2017-04-08 09:23:40
73525,4600680,208827041_70541462157,571402946_85481266703_66292,start,2017-04-08 09:28:21
73526,4600680,208827041_70541462157,571402946_85481266703_66292,step_1,2017-04-08 09:28:23


In [16]:
proc_test_clean[proc_test_clean['t_step3_conf']<0]

,client_id,visitor_id,visit_id,reached_start,reached_step_1,reached_step_2,reached_step_3,reached_confirm,completed,t_start_step1,t_step1_step2,t_step2_step3,t_step3_conf,t_total,n_back_jumps,outcome
